In [1]:
import warnings
warnings.filterwarnings("ignore")

from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
dataset = load_dataset("squad_kor_v2",trust_remote_code=True)
dataset['train']['question'][0], dataset['train']['answer'][0]

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

('드라마 예고범의 감독은 누구일까?',
 {'text': '나카무라 요시히로, 히라바야시 카츠토시, 사와다 메구미',
  'answer_start': 6302,
  'html_answer_start': 21842})

In [3]:
valid =dataset['validation']
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset['valid'] = valid

In [4]:
dataset['train']['answer'][0]

{'text': '<p>과거 개발금융시대에도 은행대출의 대부분을 재벌기업들이 가져갔으며 결과적으로 국내 중소기업의 발전이 부진했는데, 은행 설립까지 허용한다면 재벌은행의 영업 활동은 재벌기업의 사금고 역할이 될 것이라는 것이다.\n</p><p>실제로 과거 사례를 보면 은행은 아니지만 은행 이외의 금융회사가 재벌그룹 내에 소속되어 있는 경우 계열회사에 대한 안정적인 자금조달 창구 역할을 했던 것이 사실이다. 특히 투자신탁회사의 경우 모기업에 대하여 과다한 자금지원을 하다가 부실로 연결된 사례가 많이 있는데, 은행의 경우에도 이와 마찬가지일 것이라는 주장이다. 더 나아가 재벌그룹의 불법적인 자금 세탁이나 조성 등에 은행이 이용될 수 있다는 견해도 있다. 이러한 견해는 과거 재벌이라고 하는 대한민국의 거대 산업자본들이 보여준 행태가 신뢰할 수 없다는 결과에서 비롯된 것으로 볼 수 있다.\n</p>',
 'answer_start': 7398,
 'html_answer_start': 23927}

In [5]:
checkpoint = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        # print(answer)
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/66788 [00:00<?, ? examples/s]

Map:   0%|          | 0/16698 [00:00<?, ? examples/s]

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="korquad_qa_0418",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.003900,0.002069
2,0.003300,0.001970
3,0.001700,0.001900


TrainOutput(global_step=12525, training_loss=0.005260642703107657, metrics={'train_runtime': 5409.3693, 'train_samples_per_second': 37.04, 'train_steps_per_second': 2.315, 'total_flos': 5.23544633666519e+16, 'train_loss': 0.005260642703107657, 'epoch': 3.0})

In [14]:
trainer.evaluate(tokenized_dataset['valid'])

{'eval_loss': 0.0036238040775060654,
 'eval_runtime': 86.8563,
 'eval_samples_per_second': 117.032,
 'eval_steps_per_second': 7.322,
 'epoch': 3.0}